In [1]:
from pyspark.sql import SparkSession

from pyspark.sql import SQLContext

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS

from pyspark.sql.functions import udf, col, when, split, regexp_extract, sum

import numpy as np

from IPython.display import Image

from IPython.display import display

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
spark = SparkSession.builder.appName("ALS_Recommend").getOrCreate()

23/11/22 11:36:50 WARN Utils: Your hostname, nhom6nth-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.59.139 instead (on interface ens33)
23/11/22 11:36:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/22 11:36:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Tạo một đối tượng SparkContext (sc) để quản lý các tác vụ trên Spark.
sc = spark.sparkContext

# Tạo một đối tượng SQLContext (sqlContext) để thực hiện các truy vấn SQL trên Spark DataFrame.
sqlContext = SQLContext(sc)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
df_movies = spark.read.csv('movies.csv', inferSchema=True, header=True) 
df_movies.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [5]:
df_movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [6]:
df_ratings = spark.read.csv('ratings.csv', inferSchema=True, header=True) 
df_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
df_ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [8]:
df_links = spark.read.csv('links.csv', inferSchema=True, header=True) 
df_links.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- imdbId: integer (nullable = true)
 |-- tmdbId: integer (nullable = true)



In [9]:
df_links.show()

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
|      6|113277|   949|
|      7|114319| 11860|
|      8|112302| 45325|
|      9|114576|  9091|
|     10|113189|   710|
|     11|112346|  9087|
|     12|112896| 12110|
|     13|112453| 21032|
|     14|113987| 10858|
|     15|112760|  1408|
|     16|112641|   524|
|     17|114388|  4584|
|     18|113101|     5|
|     19|112281|  9273|
|     20|113845| 11517|
+-------+------+------+
only showing top 20 rows



In [10]:
# Drop rows with null values
df_linksclean = df_links.na.drop()

# Show the resulting DataFrame
df_linksclean.show()

+-------+------+------+
|movieId|imdbId|tmdbId|
+-------+------+------+
|      1|114709|   862|
|      2|113497|  8844|
|      3|113228| 15602|
|      4|114885| 31357|
|      5|113041| 11862|
|      6|113277|   949|
|      7|114319| 11860|
|      8|112302| 45325|
|      9|114576|  9091|
|     10|113189|   710|
|     11|112346|  9087|
|     12|112896| 12110|
|     13|112453| 21032|
|     14|113987| 10858|
|     15|112760|  1408|
|     16|112641|   524|
|     17|114388|  4584|
|     18|113101|     5|
|     19|112281|  9273|
|     20|113845| 11517|
+-------+------+------+
only showing top 20 rows



In [11]:
# Xóa các dòng có giá trị "(no genres listed)" theo số movieId
df_moviesclean = df_movies.filter(col('genres') != '(no genres listed)')

# Giữ lại một dòng duy nhất cho mỗi số movieId
df_moviesclean = df_moviesclean.dropDuplicates(['movieId'])

# Hiển thị PySpark DataFrame sau khi xóa
df_moviesclean.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [12]:
# Define a regular expression to extract the year
year_pattern = r"\((\d{4})\)"

# Extract the year from the 'title' column
df_moviesclean = df_moviesclean.withColumn("year", regexp_extract(df_moviesclean["title"], year_pattern, 1))

# Split the 'title' column into 'title' and 'year' columns
df_moviesclean = df_moviesclean.withColumn("title", regexp_extract(df_moviesclean["title"], "^(.*?)\s\(\d{4}\)", 1))

# Show the updated DataFrame
df_moviesclean.show(truncate=False)

+-------+------------------------------+-------------------------------------------+----+
|movieId|title                         |genres                                     |year|
+-------+------------------------------+-------------------------------------------+----+
|1      |Toy Story                     |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji                       |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men              |Comedy|Romance                             |1995|
|4      |Waiting to Exhale             |Comedy|Drama|Romance                       |1995|
|5      |Father of the Bride Part II   |Comedy                                     |1995|
|6      |Heat                          |Action|Crime|Thriller                      |1995|
|7      |Sabrina                       |Comedy|Romance                             |1995|
|8      |Tom and Huck                  |Adventure|Children                         |1995|
|9      |S

In [13]:
# Tách cột genres thành nhiều cột
max_genres = 10  # Số lượng cột genres tối đa bạn muốn tạo

# Thêm cột mới "genres_array" chứa mảng các thể loại
df_moviesclean = df_moviesclean.withColumn("genres_array", split(col("genres"), "\\|"))

# Tạo các cột genre_1, genre_2, ..., genre_max_genres
for i in range(1, max_genres + 1):
    df_moviesclean = df_moviesclean.withColumn(f"genre_{i}", when(col("genres_array").getItem(i - 1).isNotNull(), col("genres_array").getItem(i - 1)))

# Hiển thị kết quả
df_moviesclean = df_moviesclean.select(["movieId", "title", "year"] + [f"genre_{i}" for i in range(1, max_genres + 1)])
df_moviesclean.show(truncate=False)

+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|movieId|title                         |year|genre_1  |genre_2  |genre_3 |genre_4|genre_5 |genre_6|genre_7|genre_8|genre_9|genre_10|
+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|1      |Toy Story                     |1995|Adventure|Animation|Children|Comedy |Fantasy |NULL   |NULL   |NULL   |NULL   |NULL    |
|2      |Jumanji                       |1995|Adventure|Children |Fantasy |NULL   |NULL    |NULL   |NULL   |NULL   |NULL   |NULL    |
|3      |Grumpier Old Men              |1995|Comedy   |Romance  |NULL    |NULL   |NULL    |NULL   |NULL   |NULL   |NULL   |NULL    |
|4      |Waiting to Exhale             |1995|Comedy   |Drama    |Romance |NULL   |NULL    |NULL   |NULL   |NULL   |NULL   |NULL    |
|5      |Father of the Bride Part II   |1995|Comedy   |NULL     |NULL

In [14]:
# Thay thế giá trị null bằng chuỗi trống
df_moviesclean = df_moviesclean.na.fill("")

# Hiển thị kết quả
df_moviesclean.show(truncate=False)

+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|movieId|title                         |year|genre_1  |genre_2  |genre_3 |genre_4|genre_5 |genre_6|genre_7|genre_8|genre_9|genre_10|
+-------+------------------------------+----+---------+---------+--------+-------+--------+-------+-------+-------+-------+--------+
|1      |Toy Story                     |1995|Adventure|Animation|Children|Comedy |Fantasy |       |       |       |       |        |
|2      |Jumanji                       |1995|Adventure|Children |Fantasy |       |        |       |       |       |       |        |
|3      |Grumpier Old Men              |1995|Comedy   |Romance  |        |       |        |       |       |       |       |        |
|4      |Waiting to Exhale             |1995|Comedy   |Drama    |Romance |       |        |       |       |       |       |        |
|5      |Father of the Bride Part II   |1995|Comedy   |         |    

In [15]:
# Số lượng giá trị ratings trùng lặp
duplicate_count = df_moviesclean.groupBy(df_moviesclean.columns).count().filter(col("count") > 1).count()
print('Số lượng giá trị ratings trùng lặp:', duplicate_count)

# Số lượng giá trị ratings bị thiếu
missing_count = df_moviesclean.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df_moviesclean.columns]).toPandas().iloc[0].sum()
print('\nSố lượng giá trị ratings bị thiếu:', missing_count)

Số lượng giá trị ratings trùng lặp: 0

Số lượng giá trị ratings bị thiếu: 0


In [16]:
# Số lượng giá trị ratings trùng lặp
duplicate_count = df_ratings.groupBy(df_ratings.columns).count().filter(col("count") > 1).count()
print('Số lượng giá trị ratings trùng lặp:', duplicate_count)

# Số lượng giá trị ratings bị thiếu
missing_count = df_ratings.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df_ratings.columns]).toPandas().iloc[0].sum()
print('\nSố lượng giá trị ratings bị thiếu:', missing_count)

Số lượng giá trị ratings trùng lặp: 0

Số lượng giá trị ratings bị thiếu: 0


In [17]:
# Số lượng giá trị ratings trùng lặp
duplicate_count = df_linksclean.groupBy(df_linksclean.columns).count().filter(col("count") > 1).count()
print('Số lượng giá trị ratings trùng lặp:', duplicate_count)

# Số lượng giá trị ratings bị thiếu
missing_count = df_linksclean.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in df_linksclean.columns]).toPandas().iloc[0].sum()
print('\nSố lượng giá trị ratings bị thiếu:', missing_count)

Số lượng giá trị ratings trùng lặp: 0

Số lượng giá trị ratings bị thiếu: 0


In [18]:
training_df, validation_df = df_ratings.randomSplit([.8, .2])

In [19]:
# Số vòng lặp (iterations) cho thuật toán huấn luyện mô hình
iterations = 10

# Tham số regularization (chống quá mức) được sử dụng để kiểm soát độ phức tạp của mô hình và tránh overfitting
regularization_parameter = 0.1

# Rank là một tham số quan trọng liên quan đến kích thước ẩn của mô hình, thường được sử dụng trong các mô hình độ phức tạp cao như mô hình hệ thống gợi ý (recommendation systems)
rank = 4

# Một danh sách (errors) được sử dụng để lưu trữ các giá trị lỗi trong quá trình huấn luyện mô hình
errors = []

# Biến err được sử dụng để lưu trữ giá trị lỗi tạm thời trong mỗi vòng lặp
err = 0

In [20]:
# Xây dựng mô hình ALS (Alternating Least Squares) với các tham số đã được đặt trước
als = ALS(maxIter=iterations, regParam=regularization_parameter, rank=4, userCol="userId", itemCol="movieId", ratingCol="rating")

# Huấn luyện mô hình trên tập dữ liệu huấn luyện (training_df)
model = als.fit(training_df)

# Tạo dự đoán trên tập dữ liệu validation_df sử dụng mô hình đã huấn luyện
predictions = model.transform(validation_df)

# Loại bỏ các dự đoán có giá trị NaN (không phù hợp)
new_predictions = predictions.filter(col('prediction') != np.nan)

# Sử dụng đánh giá viên (evaluator) để đánh giá chất lượng dự đoán bằng cách tính Root Mean Square Error (RMSE)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(new_predictions)

# In ra giá trị RMSE, một độ đo đánh giá chất lượng của mô hình hệ thống gợi ý
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 0.8989010772560408


In [21]:
predictions.limit(10).toPandas()

,userId,movieId,rating,timestamp,prediction
0,1,296,3.0,964982967,4.957024
1,1,362,5.0,964982588,4.121805
2,1,333,5.0,964981179,4.296164
3,1,157,5.0,964984100,3.262362
4,1,235,4.0,964980908,4.310140
5,1,780,3.0,964984086,3.994410
6,1,231,5.0,964981179,3.486946
7,1,70,3.0,964982400,4.081135
8,1,151,5.0,964984041,4.088965
9,1,110,4.0,964982176,4.667264


In [22]:
predictions.join(df_moviesclean, "movieId").select("userId", "title", "year", "genre_1", "genre_2", "genre_3", "genre_4", "genre_5", "genre_6", "genre_7", "genre_8", "genre_9", "genre_10", "prediction").limit(5).toPandas()

,userId,title,year,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,prediction
0,148,Harry Potter and the Sorcerer's Stone (a.k.a. ...,2001,Adventure,Children,Fantasy,,,,,,,,3.502250
1,148,"Lord of the Rings: The Fellowship of the Ring,...",2001,Adventure,Fantasy,,,,,,,,,3.402360
2,148,Spirited Away (Sen to Chihiro no kamikakushi),2001,Adventure,Animation,Fantasy,,,,,,,,3.668964
3,148,Finding Nemo,2003,Adventure,Animation,Children,Comedy,,,,,,,3.700216
4,148,Pride & Prejudice,2005,Drama,Romance,,,,,,,,,3.643405


In [23]:
for_one_user = predictions.filter(col("userId") == 599).join(df_moviesclean, "movieId").join(df_linksclean, "movieId").select("userId", "title", "year", "genre_1", "genre_2", "genre_3", "genre_4", "genre_5", "genre_6", "genre_7", "genre_8", "genre_9", "genre_10", "tmdbId", "prediction")
for_one_user.show(5)

+------+----------------+----+---------+---------+--------+-------+-------+-------+-------+-------+-------+--------+------+----------+
|userId|           title|year|  genre_1|  genre_2| genre_3|genre_4|genre_5|genre_6|genre_7|genre_8|genre_9|genre_10|tmdbId|prediction|
+------+----------------+----+---------+---------+--------+-------+-------+-------+-------+-------+-------+--------+------+----------+
|   599|         Jumanji|1995|Adventure| Children| Fantasy|       |       |       |       |       |       |        |  8844| 2.7920644|
|   599|         Sabrina|1995|   Comedy|  Romance|        |       |       |       |       |       |       |        | 11860| 2.4248939|
|   599|Cutthroat Island|1995|   Action|Adventure| Romance|       |       |       |       |       |       |        |  1408| 2.1451287|
|   599|      Four Rooms|1995|   Comedy|         |        |       |       |       |       |       |       |        |     5| 3.0255475|
|   599|       Assassins|1995|   Action|    Crime|Thril

In [24]:
import webbrowser

link = "https://www.themoviedb.org/movie/"

for movie in for_one_user.take(2):
    movieURL = link + str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

Jumanji
Sabrina


In [25]:
for_one_user1 = predictions.filter(col("userId") == 10).join(df_moviesclean, "movieId").join(df_linksclean, "movieId").select("userId", "title", "year", "genre_1", "genre_2", "genre_3", "genre_4", "genre_5", "genre_6", "genre_7", "genre_8", "genre_9", "genre_10", "tmdbId", "prediction")
for_one_user1.show(5)

+------+--------------------+----+-------+---------+---------+--------+--------+-------+-------+-------+-------+--------+------+----------+
|userId|               title|year|genre_1|  genre_2|  genre_3| genre_4| genre_5|genre_6|genre_7|genre_8|genre_9|genre_10|tmdbId|prediction|
+------+--------------------+----+-------+---------+---------+--------+--------+-------+-------+-------+-------+--------+------+----------+
|    10|Twilight Saga: Ne...|2009|  Drama|  Fantasy|   Horror| Romance|Thriller|       |       |       |       |        | 18239|  1.599179|
|    10|     Chasing Liberty|2004| Comedy|  Romance|         |        |        |       |       |       |       |        | 14844|  4.141877|
|    10|    Incredibles, The|2004| Action|Adventure|Animation|Children|  Comedy|       |       |       |       |        |  9806| 2.9307492|
|    10|       Batman Begins|2005| Action|    Crime|     IMAX|        |        |       |       |       |       |        |   272| 3.1557572|
|    10|Bridget Jone

In [26]:
import webbrowser

link = "https://www.themoviedb.org/movie/"

for movie in for_one_user1.take(2):
    movieURL = link + str(movie.tmdbId)
    print(movie.title)
    webbrowser.open(movieURL)

Twilight Saga: New Moon, The
Chasing Liberty


In [27]:
# Tạo ra 5 gợi ý xếp hạng cho tất cả người dùng dựa trên mô hình đã huấn luyện
userRecomments = model.recommendForAllUsers(5)

# Tạo ra 5 gợi ý xếp hạng cho tất cả sản phẩm dựa trên mô hình đã huấn luyện
movieRecomments = model.recommendForAllItems(5)

In [28]:
userRecomments.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [29]:
userRecomments.select("userId", "recommendations.movieId").show(10, False)

+------+----------------------------------+
|userId|movieId                           |
+------+----------------------------------+
|1     |[3379, 6201, 4495, 2295, 33649]   |
|2     |[59018, 3379, 5075, 77846, 25906] |
|3     |[3223, 89118, 2488, 4863, 25771]  |
|4     |[7116, 4794, 7564, 6975, 2007]    |
|5     |[8477, 6818, 148881, 89759, 6201] |
|6     |[4794, 1034, 7116, 2290, 1046]    |
|7     |[4794, 1034, 7116, 2290, 1046]    |
|8     |[3379, 8477, 6201, 4495, 2295]    |
|9     |[8477, 6818, 25771, 6201, 4495]   |
|10    |[112804, 4082, 5075, 90888, 67695]|
+------+----------------------------------+
only showing top 10 rows



In [30]:
movieRecomments.select("movieId", "recommendations.userId").show(10, False)

+-------+------------------------+
|movieId|userId                  |
+-------+------------------------+
|1      |[53, 578, 12, 452, 43]  |
|3      |[53, 43, 543, 337, 93]  |
|5      |[43, 53, 543, 544, 337] |
|6      |[53, 12, 578, 452, 594] |
|9      |[543, 43, 53, 243, 492] |
|12     |[55, 406, 388, 360, 594]|
|13     |[53, 43, 543, 337, 544] |
|15     |[543, 138, 53, 584, 337]|
|16     |[53, 578, 12, 43, 250]  |
|17     |[43, 258, 53, 296, 505] |
+-------+------------------------+
only showing top 10 rows



In [32]:
users = df_ratings.select("userId").distinct().limit(3)
users.show()

+------+
|userId|
+------+
|   148|
|   463|
|   471|
+------+



In [33]:
userSubsetRecs = model.recommendForUserSubset(users, 10)

In [34]:
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[{3379, 5.0479555...|
|   463|[{3379, 5.2748623...|
|   148|[{26171, 5.321882...|
+------+--------------------+



In [35]:
userSubsetRecs.select("userId", "recommendations.movieId").show(10, False)

+------+-----------------------------------------------------------------------+
|userId|movieId                                                                |
+------+-----------------------------------------------------------------------+
|471   |[3379, 26171, 141718, 945, 33649, 59018, 77846, 25906, 89904, 6732]    |
|463   |[3379, 104339, 33649, 3200, 5607, 3746, 179135, 138966, 134796, 117531]|
|148   |[26171, 3379, 6732, 945, 81535, 173145, 59018, 44943, 67618, 94810]    |
+------+-----------------------------------------------------------------------+



In [36]:
movies = df_ratings.select("movieId").distinct().limit(3)
movies.show()

+-------+
|movieId|
+-------+
|   1580|
|   2366|
|   3175|
+-------+



In [37]:
# Tạo ra 10 gợi ý xếp hạng người dùng cho tập hợp con sản phẩm đã được chỉ định (movies) dựa trên mô hình đã huấn luyện
movieSubSetRecs = model.recommendForItemSubset(movies, 10)
# Chọn các cột "movieId" và "recommendations.userId" từ DataFrame 'movieSubSetRecs'
# Hiển thị 10 dòng đầu tiên của kết quả, trong đó "recommendations.userId" là danh sách các người dùng được gợi ý cho từng sản phẩm
movieSubSetRecs.select("movieId", "recommendations.userId").show(10, False)

+-------+------------------------------------------------+
|movieId|userId                                          |
+-------+------------------------------------------------+
|1580   |[53, 12, 452, 578, 93, 327, 276, 413, 169, 475] |
|3175   |[12, 53, 578, 594, 452, 99, 93, 169, 523, 475]  |
|2366   |[258, 154, 147, 547, 43, 296, 206, 295, 90, 505]|
+-------+------------------------------------------------+



In [38]:
# Tạo danh sách movie_ids và user_ids
movie_ids = [1580, 3175, 2366, 1590]
user_ids = [543, 543, 543, 543]

# Tạo DataFrame mới 'new_user_preds' từ danh sách movie_ids và user_ids
new_user_preds = sqlContext.createDataFrame(zip(movie_ids, user_ids), schema=["movieId", "userId"])

In [39]:
# Sử dụng mô hình để tạo ra các dự đoán xếp hạng cho DataFrame 'new_user_preds'
new_predictions = model.transform(new_user_preds)

# Hiển thị kết quả
new_predictions.show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|   1580|   543| 4.2041736|
|   3175|   543| 3.8834827|
|   2366|   543|  3.769218|
|   1590|   543| 2.5117478|
+-------+------+----------+

